In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

In [94]:
import warnings 
warnings.filterwarnings('ignore')
lookback_window = [0, 50]
number_of_clusters = 38

def cluster_composition_and_centroid(self):
## cluster composition and centroids

result = dict(zip(list(self.correlation_matrix.columns), self.apply_SPONGE())) ## composition

df_cleaned['Cluster'] = df_cleaned.index.map(result)
centroid_returns = df_cleaned.groupby('Cluster').mean() ## centroids 

df_cleaned = df_cleaned.transpose()
centroid_returns = centroid_returns.transpose()

## constituent_weights
centred_returns = df_cleaned.copy()
sigma = 0.01

constituent_weights = pd.DataFrame(index=['Weight'], columns=centred_returns.columns)
total_weight = pd.DataFrame(index=['Total weight'], columns=[i for i in range(number_of_clusters)], data=np.zeros((1, number_of_clusters)))

## we first compute the difference between the cluster centroid return and the cluster ticker return
for ticker in centred_returns.columns:
    centred_returns[ticker][:-1] = centred_returns[ticker][:-1] - centroid_returns[int(centred_returns[ticker]['Cluster'])]

## we use this difference to compute the distance between each asset and its cluster centroid return 
for ticker in centred_returns.columns:
    constituent_weights[ticker] = np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)
    total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight'] += np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)

## we normalize the weights
for ticker in centred_returns.columns:
    constituent_weights[ticker] = constituent_weights[ticker]['Weight']/total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight']

## check whether the weights equal to 0 within each cluster: 
# constituent_weights[[ticker for ticker in centred_returns.columns if centred_returns[ticker]['Cluster']  == 1.0]].sum(axis=1)
    
## cluster returns 
    
cluster_returns = pd.DataFrame(index=df_cleaned.index[:-1], columns=np.arange(number_of_clusters), data=np.zeros((df_cleaned.shape[0] - 1, number_of_clusters)))

for ticker in df_cleaned.columns:
    cluster_returns[int(df_cleaned[ticker]['Cluster'])] = cluster_returns[int(df_cleaned[ticker]['Cluster'])] + constituent_weights[ticker]['Weight'] * df_cleaned[ticker][:-1]

IndentationError: expected an indented block after function definition on line 6 (2352418720.py, line 9)

In [7]:
from pypfopt.efficient_frontier import EfficientFrontier
import process
from PyFolioCC import PyFolioCC 

historical_data = df_cleaned
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
# cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
# markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)

In [8]:
constituent_weights

[['cluster 1',
  [['HNI', 0.19798917174162717],
   ['HRL', 0.12410558494131475],
   ['HSY', 0.14181363553163243],
   ['KFY', 0.049055668000294976],
   ['ROL', 0.20520724283043798],
   ['TEO', 0.1414601594168903],
   ['WCC', 0.14036853753780237]]],
 ['cluster 2',
  [['AZN', 0.17677429874786893],
   ['BCE', 0.04465962691058011],
   ['BCS', 0.09381523793087525],
   ['CLS', 0.01010696504230512],
   ['EFX', 0.08883157819393185],
   ['FC', 0.01709162046001437],
   ['GFF', 0.04192120421726353],
   ['JBL', 0.061789361226571395],
   ['KEP', 0.05685915737745913],
   ['KR', 0.06495940694087857],
   ['MAS', 0.042074656445308056],
   ['PKX', 0.012101164506693993],
   ['PRGO', 0.05356909007851171],
   ['PWR', 0.04041357559827461],
   ['REG', 0.15659407007553236],
   ['YUM', 0.038438986247931156]]],
 ['cluster 3',
  [['AFL', 0.015633181320684416],
   ['AIG', 0.038179792595482494],
   ['ALL', 0.028584488127078526],
   ['AXP', 0.02205549013298679],
   ['BAC', 0.03177838125063279],
   ['BEN', 0.01430235

In [60]:
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)
final_weights = process.final_weights(markowitz_weights=markowitz_weights_res, constituent_weights=constituent_weights)
print(corr_matrix.shape, len(cluster_comp), len(constituent_weights), len(cluster_return), len(markowitz_weights_res), len(final_weights))


(695, 695) 37 37 37 37 692


In [82]:
cluster_comp

[['cluster 1',
  ['ADI',
   'ARW',
   'ASG',
   'BIO',
   'CLF',
   'COO',
   'CWT',
   'DLX',
   'EFX',
   'ENZ',
   'EV',
   'GES',
   'GGT',
   'GLT',
   'HVT',
   'IEX',
   'KMT',
   'LLY',
   'MCD',
   'MDC',
   'MLI',
   'MTW',
   'NLY',
   'NOC',
   'NVR',
   'O',
   'RSG',
   'SMG',
   'SPG',
   'TEF',
   'TG',
   'TOL',
   'TTC',
   'UFI',
   'UNFI',
   'VAR',
   'VSH',
   'WEN',
   'WSM',
   'WSO',
   'WTS',
   'XLK'],
  array([-0.03, -0.02,  0.01, -0.  ,  0.02,  0.01, -0.  , -0.  ,  0.01,
          0.01, -0.  ,  0.01, -0.  , -0.  , -0.04,  0.  , -0.01, -0.  ,
         -0.  , -0.01, -0.  ,  0.  , -0.  ,  0.01, -0.  , -0.01, -0.01,
         -0.  , -0.01, -0.02, -0.  ,  0.01,  0.01, -0.03,  0.  ,  0.01,
         -0.01, -0.01,  0.  ,  0.01, -0.  , -0.01,  0.01,  0.01, -0.03,
          0.  ,  0.02, -0.  , -0.01, -0.02])],
 ['cluster 2',
  ['AIV',
   'BCE',
   'BDN',
   'BRK',
   'CDE',
   'CLS',
   'DHI',
   'GF',
   'GIB',
   'GLW',
   'HIW',
   'IRM',
   'JBL',
   'MCA',
   'MC